In [3]:
import torch
from run import selectModel
from torchvision.models import AlexNet
from torch.autograd import Variable
import torchvision
from torchsummary import summary

In [5]:
model_path = './checkpoints/SGD_l1_mobilenetv1_chestX_10_edi_2.0_theta_10_lam_1.0e-04.pt'
checkpoint = torch.load(model_path,map_location='cpu')
model = selectModel('mobilenetv1')
model.load_state_dict = checkpoint['model_state_dict']

modelName: MobileNetV1


In [7]:
from graphviz import Digraph
import torch
from torch.autograd import Variable


def make_dot(var, params=None):
    """ Produces Graphviz representation of PyTorch autograd graph

    Blue nodes are the Variables that require grad, orange are Tensors
    saved for backward in torch.autograd.Function

    Args:
        var: output Variable
        params: dict of (name, Variable) to add names to node that
            require grad (TODO: make optional)
    """
    if params is not None:
        assert isinstance(params.values()[0], Variable)
        param_map = {id(v): k for k, v in params.items()}

    node_attr = dict(style='filled',
                     shape='box',
                     align='left',
                     fontsize='12',
                     ranksep='0.1',
                     height='0.2')
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()

    def size_to_str(size):
        return '('+(', ').join(['%d' % v for v in size])+')'

    def add_nodes(var):
        if var not in seen:
            if torch.is_tensor(var):
                dot.node(str(id(var)), size_to_str(var.size()), fillcolor='orange')
            elif hasattr(var, 'variable'):
                u = var.variable
                name = param_map[id(u)] if params is not None else ''
                node_name = '%s\n %s' % (name, size_to_str(u.size()))
                dot.node(str(id(var)), node_name, fillcolor='lightblue')
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, 'next_functions'):
                for u in var.next_functions:
                    if u[0] is not None:
                        dot.edge(str(id(u[0])), str(id(var)))
                        add_nodes(u[0])
            if hasattr(var, 'saved_tensors'):
                for t in var.saved_tensors:
                    dot.edge(str(id(t)), str(id(var)))
                    add_nodes(t)
    add_nodes(var.grad_fn)
    return dot

if __name__ == '__main__':
    from torchvision.models import AlexNet
    from torch.autograd import Variable
    import torch

    # net = AlexNet()
    x = Variable(torch.randn((32,1,960,960)))#out_channel, input channel, image_size length*height
    y = model(x)
    g = make_dot(y)
    g.view()
    pass

In [6]:
       #使用 pip install torchsummary
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
a = summary(model, input_size=(1, 192*5, 192*5)) # image size 1 channel,192*5 * 192*5

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 48, 48]             288
       BatchNorm2d-2           [-1, 32, 48, 48]              64
            Conv2d-3           [-1, 32, 48, 48]             288
       BatchNorm2d-4           [-1, 32, 48, 48]              64
            Conv2d-5           [-1, 64, 48, 48]           2,048
       BatchNorm2d-6           [-1, 64, 48, 48]             128
             Block-7           [-1, 64, 48, 48]               0
            Conv2d-8           [-1, 64, 24, 24]             576
       BatchNorm2d-9           [-1, 64, 24, 24]             128
           Conv2d-10          [-1, 128, 24, 24]           8,192
      BatchNorm2d-11          [-1, 128, 24, 24]             256
            Block-12          [-1, 128, 24, 24]               0
           Conv2d-13          [-1, 128, 24, 24]           1,152
      BatchNorm2d-14          [-1, 128,